In [1]:
import numpy as np

# environment parameters
num_states = 10
num_actions = 2

# replay buffer parameters
buffer_size = 1000
buffer_index = 0
replay_buffer = np.zeros((buffer_size, 4))  # state, action, reward, next_state

# Q-network parameters
q_network = np.zeros((num_states, num_actions))

# training parameters
epsilon = 0.1
max_timesteps = 100
batch_size = 32
learning_rate = 0.1
discount_factor = 0.99
num_episodes = 1000

# training loop
for episode in range(num_episodes):
    state = 0
    goal = num_states - 1
    episode_buffer = []
    timestep = 0
    done = False
    
    while not done and timestep < max_timesteps:
        # choose action based on Q-network and exploration rate
        if np.random.random() < epsilon:
            action = np.random.randint(num_actions)
        else:
            print(state, q_network[state])
            action = np.argmax(q_network[state])
        
        # take action and observe next state and reward
        if action == 0:
            next_state = state - 1
            reward = -1
        else:
            next_state = state + 1
            reward = 1
        
        # add transition to episode buffer and replay buffer
        episode_buffer.append((state, action, reward, next_state))
        replay_buffer[buffer_index] = (state, action, reward, next_state)
        buffer_index = (buffer_index + 1) % buffer_size
        
        # update Q-network with minibatch of transitions
        if buffer_index >= batch_size:
            minibatch = replay_buffer[np.random.choice(buffer_size, batch_size)]
            for transition in minibatch:
                s, a, r, s_prime = transition
                target = r + discount_factor * np.max(q_network[s_prime])
                q_network[s, a] = (1 - learning_rate) * q_network[s, a] + learning_rate * target
        
        # update state and timestep
        state = next_state
        timestep += 1
        
        # check if episode is done
        if state == goal or timestep == max_timesteps:
            done = True
            
            # update replay buffer with hindsight replay
            for i in range(len(episode_buffer)):
                s, a, r, s_prime = episode_buffer[i]
                new_reward = -1 if i < len(episode_buffer) - 1 else reward
                new_transition = (s, a, new_reward, s_prime)
                replay_buffer[buffer_index] = new_transition
                buffer_index = (buffer_index + 1) % buffer_size

[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]


IndexError: index -11 is out of bounds for axis 0 with size 10